### Image pre-process

In [1]:
import os
import csv

import cv2
import numpy as np
import sklearn
import sklearn.utils


samples = []
#with open('data/driving_log.csv') as csvfile:
#    reader = csv.reader(csvfile)
#    for line in reader:
#        samples.append(line)
        
with open('data2/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)        

samples = sklearn.utils.shuffle(samples)        
        

#from sklearn.model_selection import train_test_split
#train_samples, validation_samples = train_test_split(samples, test_size=0.2)


def generator(samples, batch_size=32):
    
    batch_size = int(batch_size/6)
    
    num_samples = len(samples)
    #samples = sklearn.utils.shuffle(samples)
    for offset in range(0, batch_size):
        batch_samples = samples[offset:offset+batch_size]

        images = []
        angles = []
        for batch_sample in batch_samples:
            for i in range(3):
                #name = 'data\\IMG\\'+batch_sample[i].split('\\')[-1]
                
                name =  batch_sample[i].split('\\')[-3] + '\\' + \
                batch_sample[i].split('\\')[-2] + '\\' + batch_sample[i].split('\\')[-1]

                center_image = cv2.imread(name)
                b,g,r = cv2.split(center_image)
                img2 = cv2.merge([r,g,b])
                if i == 0:
                    center_angle = float(batch_sample[3])
                if i == 1:
                    center_angle = float(batch_sample[3]) + 0.25            
                if i == 2:
                    center_angle = float(batch_sample[3]) - 0.25   
                
                images.append(img2)
                angles.append(center_angle)
                
        augmented_images, augmented_angles = [], []   
        
        for image, angle in zip(images,angles):
            augmented_images.append(image)
            augmented_angles.append(angle)
            augmented_images.append(cv2.flip(image,1))
            augmented_angles.append(angle*-1.0)
            
        X_train = np.array(augmented_images)
        y_train = np.array(augmented_angles)
    
        
        return sklearn.utils.shuffle(X_train, y_train)

In [2]:
X_train, y_train = generator(samples, batch_size=30000)

### NVidia Network

In [3]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Cropping2D, Dropout
from keras.layers import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Conv2D(24,(5,5),strides=(2,2),activation='relu', padding='valid', kernel_initializer='he_normal'))
model.add(Conv2D(36,(5,5),strides=(2,2),activation='relu', padding='valid', kernel_initializer='he_normal'))
model.add(Conv2D(48,(5,5),strides=(2,2),activation='relu', padding='valid', kernel_initializer='he_normal'))
model.add(Conv2D(64,(3,3),activation='relu', padding='valid', kernel_initializer='he_normal'))
model.add(Conv2D(64,(3,3),activation='relu', padding='valid', kernel_initializer='he_normal'))
model.add(Flatten())
model.add(Dense(1164, kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(100, kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(50, kernel_initializer='he_normal'))
model.add(Dense(10, kernel_initializer='he_normal'))
model.add(Dense(1, kernel_initializer='he_normal'))

model.compile(loss='mse', optimizer='adam')


Using TensorFlow backend.


In [5]:
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)
model.save('model_lane2.h5')

Train on 24000 samples, validate on 6000 samples
Epoch 1/3
24000/24000 [==============================] - 70s - loss: 0.0500 - val_loss: 0.0486
Epoch 2/3
24000/24000 [==============================] - 71s - loss: 0.0455 - val_loss: 0.0451
Epoch 3/3
24000/24000 [==============================] - 71s - loss: 0.0430 - val_loss: 0.0457


In [4]:
from keras.models import load_model

model = load_model('model_lane2.h5')



Using TensorFlow backend.
